

## **Dataset Overview: CIFAR-10**

Before diving into the implementations, here's a brief overview of the CIFAR-10 dataset:

- **Number of Classes:** 10 (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck)
- **Image Size:** 32x32 pixels, 3 channels (RGB)
- **Training Samples:** 50,000
- **Testing Samples:** 10,000

The dataset is commonly used for image classification tasks and provides a standardized benchmark for evaluating different machine learning models.




## **1. TensorFlow Implementation**

### **Step-by-Step Code with Detailed Comments**



In [ ]:
# TensorFlow Implementation for CIFAR-10 Classification

# Step 1: Import Necessary Libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10

# Step 2: Load and Preprocess the CIFAR-10 Dataset
# Load the dataset; it returns training and testing data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1 by dividing by 255
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Step 3: Define the CNN Model Architecture using Sequential API
model = models.Sequential([
    # First Convolutional Layer:
    # - 32 filters
    # - 3x3 kernel size
    # - ReLU activation
    # - Input shape matches CIFAR-10 images (32x32x3)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    
    # First Max Pooling Layer:
    # - 2x2 pool size reduces spatial dimensions
    layers.MaxPooling2D((2, 2)),
    
    # Second Convolutional Layer:
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    # Second Max Pooling Layer:
    layers.MaxPooling2D((2, 2)),
    
    # Third Convolutional Layer:
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    # Flatten the 3D feature maps to 1D feature vectors
    layers.Flatten(),
    
    # Fully Connected Layer with 64 units and ReLU activation
    layers.Dense(64, activation='relu'),
    
    # Output Layer with 10 units (one for each class) and softmax activation
    layers.Dense(10, activation='softmax')
])

# Step 4: Compile the Model
model.compile(optimizer='adam',                      # Adam optimizer for training
              loss='sparse_categorical_crossentropy',# Loss function suitable for integer labels
              metrics=['accuracy'])                  # Metric to monitor during training

# Step 5: Train the Model
model.fit(train_images, train_labels, 
          epochs=10,                                # Number of epochs
          batch_size=64,                            # Number of samples per gradient update
          validation_data=(test_images, test_labels))  # Data for validating the model

# Step 6: Evaluate the Model on Test Data
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


### **Explanation of TensorFlow Code**

1. **Import Libraries:** TensorFlow and Keras modules are imported to build and train the CNN.

2. **Load and Preprocess Data:** The CIFAR-10 dataset is loaded, and pixel values are normalized to improve training performance.

3. **Define Model Architecture:** A Sequential model is constructed with three convolutional layers interleaved with max-pooling layers, followed by a flattening layer and two dense layers. The final dense layer uses softmax activation for multi-class classification.

4. **Compile the Model:** The model is compiled with the Adam optimizer, using sparse categorical cross-entropy as the loss function since the labels are integers. Accuracy is set as the metric to monitor.

5. **Train the Model:** The model is trained for 10 epochs with a batch size of 64. Validation data is provided to monitor performance on unseen data during training.

6. **Evaluate the Model:** After training, the model's performance is evaluated on the test dataset, and the test accuracy is printed.

---

## **2. PyTorch Implementation**

### **Step-by-Step Code with Detailed Comments**

In [ ]:
# PyTorch Implementation for CIFAR-10 Classification

# Step 1: Import Necessary Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Step 2: Define Transformations for Data Augmentation and Normalization
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL images to tensors
    transforms.Normalize((0.5, 0.5, 0.5),  # Normalize each channel (R, G, B) to have mean=0.5
                         (0.5, 0.5, 0.5))  # and standard deviation=0.5
])

# Step 3: Load the CIFAR-10 Training and Testing Datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

# Step 4: Define the CNN Model Architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # First Convolutional Layer:
        # - 32 filters
        # - 3x3 kernel size
        # - Stride of 1
        self.conv1 = nn.Conv2d(3, 32, 3, stride=1, padding=1)
        
        # Second Convolutional Layer:
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        
        # Third Convolutional Layer:
        self.conv3 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
        
        # Max Pooling Layer with 2x2 kernel
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully Connected Layer with 64 units
        self.fc1 = nn.Linear(64 * 4 * 4, 64)  # CIFAR-10 images are 32x32. After three poolings, size is 4x4
        
        # Output Layer with 10 units (one for each class)
        self.fc2 = nn.Linear(64, 10)
        
        # Activation Function
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Apply first convolutional layer followed by ReLU activation and pooling
        x = self.pool(self.relu(self.conv1(x)))
        
        # Apply second convolutional layer followed by ReLU activation and pooling
        x = self.pool(self.relu(self.conv2(x)))
        
        # Apply third convolutional layer followed by ReLU activation and pooling
        x = self.pool(self.relu(self.conv3(x)))
        
        # Flatten the tensor into a vector for the fully connected layers
        x = x.view(-1, 64 * 4 * 4)
        
        # Apply first fully connected layer followed by ReLU activation
        x = self.relu(self.fc1(x))
        
        # Apply output layer
        x = self.fc2(x)
        
        return x

# Step 5: Instantiate the Model, Define Loss Function and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
model = CNN().to(device)  # Move model to the selected device

criterion = nn.CrossEntropyLoss()  # Suitable loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001

# Step 6: Train the Model
num_epochs = 10  # Number of epochs to train

for epoch in range(num_epochs):  # Loop over the dataset multiple times
    running_loss = 0.0
    model.train()  # Set model to training mode
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the selected device
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass (compute gradients)
        optimizer.step()  # Update weights
        
        running_loss += loss.item()
        
    # Print average loss for this epoch
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader):.4f}")

print('Finished Training')

# Step 7: Evaluate the Model on Test Data
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)  # Move data to the selected device
        outputs = model(images)  # Forward pass
        _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")




### **Explanation of PyTorch Code**

1. **Import Libraries:** PyTorch and torchvision libraries are imported for building and training the CNN.

2. **Data Transformations:** The images are converted to tensors and normalized. Normalization helps in speeding up the training and achieving better performance.

3. **Load Datasets:** The CIFAR-10 training and testing datasets are loaded using `torchvision.datasets.CIFAR10`. Data loaders are created to handle batching and shuffling.

4. **Define Model Architecture:** A custom CNN class is defined by subclassing `nn.Module`. It includes three convolutional layers with ReLU activations and max-pooling, followed by two fully connected layers.

5. **Instantiate Model, Define Loss and Optimizer:** The model is moved to the GPU if available. Cross-entropy loss is used for multi-class classification, and the Adam optimizer is selected for training.

6. **Train the Model:** The training loop iterates over the dataset for a specified number of epochs. For each batch, it performs a forward pass, computes the loss, performs backpropagation, and updates the weights.

7. **Evaluate the Model:** After training, the model is evaluated on the test dataset. The accuracy is computed by comparing the predicted labels with the true labels.

---




# **3. Keras Implementation (with TensorFlow Backend)**
 **Step-by-Step Code with Detailed Comments**


In [ ]:
# Keras Implementation for CIFAR-10 Classification

# Step 1: Import Necessary Libraries
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Step 2: Load and Preprocess the CIFAR-10 Dataset
# Load the dataset; it returns training and testing data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1 by dividing by 255
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Optional: Convert class vectors to binary class matrices (one-hot encoding)
# Not necessary if using 'sparse_categorical_crossentropy' loss
# train_labels = to_categorical(train_labels, 10)
# test_labels = to_categorical(test_labels, 10)

# Step 3: Define the CNN Model Architecture using Sequential API
model = Sequential([
    # First Convolutional Layer:
    # - 32 filters
    # - 3x3 kernel size
    # - ReLU activation
    # - Input shape matches CIFAR-10 images (32x32x3)
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    
    # First Max Pooling Layer:
    # - 2x2 pool size reduces spatial dimensions
    MaxPooling2D((2, 2)),
    
    # Second Convolutional Layer:
    Conv2D(64, (3, 3), activation='relu'),
    
    # Second Max Pooling Layer:
    MaxPooling2D((2, 2)),
    
    # Third Convolutional Layer:
    Conv2D(64, (3, 3), activation='relu'),
    
    # Flatten the 3D feature maps to 1D feature vectors
    Flatten(),
    
    # Fully Connected Layer with 64 units and ReLU activation
    Dense(64, activation='relu'),
    
    # Output Layer with 10 units (one for each class) and softmax activation
    Dense(10, activation='softmax')
])

# Step 4: Compile the Model
model.compile(optimizer='adam',                      # Adam optimizer for training
              loss='sparse_categorical_crossentropy',# Loss function suitable for integer labels
              metrics=['accuracy'])                  # Metric to monitor during training

# Step 5: Train the Model
model.fit(train_images, train_labels, 
          epochs=10,                                # Number of epochs
          batch_size=64,                            # Number of samples per gradient update
          validation_data=(test_images, test_labels))  # Data for validating the model

# Step 6: Evaluate the Model on Test Data
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_acc * 100:.2f}%")



### **Explanation of Keras Code**

1. **Import Libraries:** Keras modules are imported to build and train the CNN. Keras is integrated with TensorFlow as its backend.

2. **Load and Preprocess Data:** The CIFAR-10 dataset is loaded, and pixel values are normalized to improve training performance. Optionally, labels can be one-hot encoded, but it's not necessary when using `sparse_categorical_crossentropy` loss.

3. **Define Model Architecture:** A Sequential model is constructed with three convolutional layers interleaved with max-pooling layers, followed by a flattening layer and two dense layers. The final dense layer uses softmax activation for multi-class classification.

4. **Compile the Model:** The model is compiled with the Adam optimizer, using sparse categorical cross-entropy as the loss function since the labels are integers. Accuracy is set as the metric to monitor.

5. **Train the Model:** The model is trained for 10 epochs with a batch size of 64. Validation data is provided to monitor performance on unseen data during training.

6. **Evaluate the Model:** After training, the model's performance is evaluated on the test dataset, and the test accuracy is printed.


---

## **Summary and Comparison**

| **Aspect**             | **TensorFlow**                                       | **PyTorch**                                     | **Keras (TensorFlow)**                          |
|------------------------|------------------------------------------------------|-------------------------------------------------|-------------------------------------------------|
| **Model Definition**   | Sequential API with Keras layers                      | Custom `nn.Module` class with explicit layers   | Sequential API with Keras layers                 |
| **Training Process**   | High-level `model.fit()` method                        | Manual training loop with forward and backward passes | High-level `model.fit()` method                |
| **Data Handling**      | `tf.data` pipelines and built-in data augmentation      | `torchvision.datasets` and `DataLoader`         | Keras data generators and built-in preprocessing |
| **Flexibility**        | Highly flexible with TensorFlow's extensive features    | Extremely flexible with dynamic computation graphs | Moderate flexibility, easy to use               |
| **Ease of Use**        | Intermediate; requires understanding of TensorFlow      | Requires more boilerplate code                  | Very user-friendly and easy to prototype         |
| **Performance**        | Optimized for production and scalability                | Preferred for research and experimentation      | Excellent for rapid development and prototyping   |

### **Key Takeaways**

- **TensorFlow** offers a balance between flexibility and ease of use, making it suitable for both research and production environments.
  
- **PyTorch** provides unparalleled flexibility and control, making it the go-to choice for researchers and those who require dynamic computation graphs.
  
- **Keras** (with TensorFlow backend) is the most user-friendly option, ideal for beginners and rapid prototyping due to its simple and intuitive API.


